# MLFlow Tracking Server For Anaconda Enterprise

Before performing an installation you should familiarize yourself with below topics:

* [High Level Concepts](https://github.com/shapeandshare/mlflow.tracking.server/blob/main/docs/source/high_level_concepts.md)
* [MLFlow Tracking Server Overview](https://github.com/shapeandshare/mlflow.tracking.server/blob/main/docs/source/server_overview.md)
* [Installation Guide](https://github.com/shapeandshare/mlflow.tracking.server/blob/main/docs/source/installation_guide.md)

### 1. Create an instance of an AE Client

In [ ]:
from anaconda.enterprise.server.sdk.client import AEClient
from src.utils import get_ae_client

client: AEClient = get_ae_client()

### 2. Populate our service account secrets used for configuration (Part 1)

In [ ]:
from anaconda.enterprise.server.common.sdk import demand_env_var

# Define our secrets
secret_names: list[str] = ["MLFLOW_BACKEND_STORE_URI", "MLFLOW_ARTIFACTS_DESTINATION", "MLFLOW_TRACKING_GC_TTL"]

# Remove possibly pre-existing configurations
for name in secret_names:
    try:
        client.secret_delete(key=name)
    except Exception as error:
        # This is broad to allow for catching and reporting everything.
        print(error)

    # [re]create the secrets using the latest values.
    value: str = demand_env_var(name=name)
    print(f"{name}:{value}")
    client.secret_put(key=name, value=value)

### 3. Upload Project

In [ ]:
from pathlib import Path
from anaconda.enterprise.server.contracts import ProjectUploadResponse

# Define the server template to upload
SERVER_TEMPLATE_PATH: str = Path("mlflow.tracking.server-0.5.12.tar.gz").resolve().as_posix()

# Define the project name
SERVER_PROJECT_NAME: str = "dev.mlflow.tracking.server"

upload_response: ProjectUploadResponse = client.project_upload(
    project_archive_path=SERVER_TEMPLATE_PATH,
    name=SERVER_PROJECT_NAME,
)
upload_response

### 3.5 (Disaster Recovery Steps)
If performing a disaster recover then at this point the data can be restored to database and filesystem before moving to step 4.

### 4. Get project revisions

In [ ]:
project_revisions = client.project_revisions_get(project_id=upload_response.id)

### 5. Define the Deployment Options

In [ ]:
SERVER_DEPLOYMENT_NAME: str = "dev-mlflow-tracking-server"
SERVER_DEPLOYMENT_STATIC_ENDPOINT: str = "dev-mlflow-tracking-server"

deploy_params: dict = {
    "project_id": upload_response.id,
    "deployment_name": SERVER_DEPLOYMENT_NAME,
    "revision_id": project_revisions[0].id,
    "command": project_revisions[0].commands[0].id,
    "variables": {
        "MLFLOW_BACKEND_STORE_URI": demand_env_var(name="MLFLOW_BACKEND_STORE_URI"),
        "MLFLOW_ARTIFACTS_DESTINATION": demand_env_var(name="MLFLOW_ARTIFACTS_DESTINATION"),
        "MLFLOW_TRACKING_GC_TTL": demand_env_var(name="MLFLOW_TRACKING_GC_TTL"),
    },
    "static_endpoint": SERVER_DEPLOYMENT_STATIC_ENDPOINT,
}
deploy_params

### 6. Deploy the MLFlow Tracking Server

In [ ]:
from anaconda.enterprise.server.contracts import ProjectDeployResponse

project_deploy_response: ProjectDeployResponse = client.project_deploy(**deploy_params)
project_deploy_response

### 7. Generate Private Access Token

In [ ]:
access_token: str = client.deployment_token_get(deployment_id=project_deploy_response.id)
access_token

### 8. Get Deployment URL

In [ ]:
service_endpoint: str = project_deploy_response.url
service_endpoint

### 9. Populate our service account secrets used for configuration (Part 2)

In [ ]:
# Define our secrets
secrets: dict[str, str] = {
    "MLFLOW_TRACKING_URI": service_endpoint,
    "MLFLOW_REGISTRY_URI": service_endpoint,
    "MLFLOW_TRACKING_TOKEN": access_token,
}

print("The below values MUST bew provided to clients who wish to access the server and it's API.")
print(
    "Please note that since this is a private deployment that a new token MUST be provided each time the server is restarted."
)

# [Re]create the secrets using the latest values.
for key, value in secrets.items():
    print(f"{key}:{value}")
    client.secret_delete(key=key)
    client.secret_put(key=key, value=value)

### 10. Create the Garbage Collection Schedule

It is recommended to create a garbage collection schedule.  To do so use the ae5 tools.  The below will generate an ae5 command suitable for creating a basic schedule:

In [ ]:
job_creation_command: str = f"ae5 job create --command 'GarbageCollection' --schedule '*/10 * * * *' --name 'Scheduled {SERVER_PROJECT_NAME} Garbage Collection' '{SERVER_PROJECT_NAME}'"
job_creation_command